In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


2024-04-28 14:21:35.590825: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 14:21:35.590852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 14:21:35.591796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 14:21:35.597190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 14:21:36.318614: W tensorflow/compiler/tf2

In [8]:
# Datos de ejemplo (secuencia de números)
df = pd.read_excel('sector8TS.xls')  # Reemplaza 'datos.xlsx' con el nombre de tu archivo Excel

In [9]:
df.head()

,datos
0,0.217553
1,0.211681
2,0.181088
3,0.181397
4,0.165637


In [10]:
data = df['datos'].values

In [11]:
print(data)

[0.21755263 0.21168112 0.1810878  0.18139689 0.16563669 0.10352298
 0.0936341  0.08868985 0.0936341  0.20673669 0.48485779 0.67058094
 0.69128558 0.77317667 0.5877626  0.43881336 0.41841781 0.45982689
 0.43912244 0.38195298 0.43325094 0.43356002 0.35661318 0.19623001
 0.13473428 0.1093943  0.05191594 0.04202725 0.07323856 0.09826946
 0.21199021 0.34579724 0.55716928 0.69592093 0.85568593 0.83034613
 0.69622983 0.62917186 0.61835592 0.65451166 0.67552538 0.44839315
 0.44406669 0.44375779 0.5        0.52132262 0.30469724 0.20673669
 0.14462297 0.05747836 0.02101372 0.01112484 0.04758967 0.05778745
 0.20673669 0.36093945 0.58281835 0.62391834 0.67521629 0.62391834
 0.56180463 0.47435112 0.5466626  0.54758967 0.4749691  0.45395557
 0.44901113 0.37669964 0.50556242 0.55222502 0.29913482 0.2527813
 0.17583447 0.06211371 0.01606928 0.03646483 0.05840543 0.04697169
 0.14462297 0.30407907 0.54140908 0.63967854 0.65451166 0.60815814
 0.50494444 0.57787391 0.5469715  0.66563651 0.50494444 0.46971

In [12]:
# Preparar datos para LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float32), tf.convert_to_tensor(y, dtype=tf.float32)


In [13]:
# Definir hiperparámetros
n_steps = 4
n_features = 1

# Preparar datos
X, y = prepare_data(data, n_steps)

2024-04-28 14:23:30.265391: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 14:23:30.299614: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 14:23:30.299873: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [14]:
print(X)


tf.Tensor(
[[0.21755263 0.21168113 0.1810878  0.18139689]
 [0.21168113 0.1810878  0.18139689 0.16563669]
 [0.1810878  0.18139689 0.16563669 0.10352298]
 ...
 [0.49042022 0.52626705 0.5055624  0.6288628 ]
 [0.52626705 0.5055624  0.6288628  0.62391835]
 [0.5055624  0.6288628  0.62391835 0.5877626 ]], shape=(451, 4), dtype=float32)


In [15]:
print(y)

tf.Tensor(
[0.16563669 0.10352298 0.09363411 0.08868986 0.09363411 0.20673668
 0.4848578  0.6705809  0.69128555 0.77317667 0.5877626  0.43881336
 0.4184178  0.4598269  0.43912244 0.38195297 0.43325093 0.43356
 0.3566132  0.19623001 0.13473429 0.1093943  0.05191594 0.04202725
 0.07323856 0.09826946 0.2119902  0.34579724 0.55716926 0.69592094
 0.85568595 0.8303461  0.6962298  0.62917185 0.61835593 0.6545117
 0.67552537 0.44839314 0.4440667  0.4437578  0.5        0.5213226
 0.30469725 0.20673668 0.14462297 0.05747836 0.02101372 0.01112484
 0.04758967 0.05778745 0.20673668 0.36093944 0.5828183  0.62391835
 0.6752163  0.62391835 0.56180465 0.4743511  0.5466626  0.54758966
 0.4749691  0.45395556 0.44901112 0.37669966 0.5055624  0.55222505
 0.29913482 0.2527813  0.17583446 0.06211371 0.01606928 0.03646483
 0.05840543 0.04697169 0.14462297 0.3040791  0.5414091  0.63967854
 0.6545117  0.6081582  0.50494444 0.5778739  0.5469715  0.6656365
 0.50494444 0.46971577 0.49505556 0.4128554  0.40760204 0

In [16]:
# Definir el modelo
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [17]:
# Entrenar el modelo
history = model.fit(X, y, epochs=200, verbose=1)




Epoch 1/200


2024-04-28 14:23:43.358076: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3a11a22a00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-28 14:23:43.358100: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2024-04-28 14:23:43.362770: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-28 14:23:43.376638: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1714339423.432728   51797 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15/15 [==============================] - 2s 10ms/step - loss: 0.1535
Epoch 2/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0880
Epoch 3/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0429
Epoch 4/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0333
Epoch 5/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0319
Epoch 6/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0305
Epoch 7/200
15/15 [==============================] - 0s 4ms/step - loss: 0.0294
Epoch 8/200
15/15 [==============================] - 0s 4ms/step - loss: 0.0279
Epoch 9/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0268
Epoch 10/200
15/15 [==============================] - 0s 4ms/step - loss: 0.0254
Epoch 11/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0240
Epoch 12/200
15/15 [==============================] - 0s 5ms/step - loss: 0.0223
Epoch 13/200
15/15 [============================

In [18]:
# Hacer predicciones
x_input = tf.convert_to_tensor([[i] for i in range(100, 104)])
x_input = tf.reshape(x_input, (1, n_steps, n_features))
y_pred = model.predict(x_input, verbose=0)

print("Predicción:", y_pred[0][0])

Predicción: -0.56533706
